In [1]:
# import libraries
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Load the point cloud
pcd = o3d.io.read_point_cloud("pc_assignment\shoe_pc.ply")

# Function to create a coordinate frame
def create_coordinate_frame(size=0.8, origin=[0, 0, 0]):
    return o3d.geometry.TriangleMesh.create_coordinate_frame(size=size, origin=origin)

# Visualize the initial point cloud with coordinate frame
print("Displaying the initial point cloud...")
o3d.visualization.draw_geometries([pcd, create_coordinate_frame()])

Displaying the initial point cloud...


In [4]:

# Estimate the ground plane using RANSAC
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model

# Visualize the plane inliers (floor points)
floor_points = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

print("Displaying the floor points and remaining points...")
floor_points.paint_uniform_color([1.0, 0, 0])  # Red color for floor points
o3d.visualization.draw_geometries([floor_points, outlier_cloud, create_coordinate_frame()])


Displaying the floor points and remaining points...


In [5]:
# Compute the normal vector to the plane
normal_vector = np.array([a, b, c])

# Compute the rotation axis (cross product with x-axis)
x_axis = np.array([1, 0, 0])
rotation_axis = np.cross(normal_vector, x_axis)
rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)

# Compute the angle between the normal vector and x-axis
theta = np.arccos(np.dot(normal_vector, x_axis) / np.linalg.norm(normal_vector))

# Compute the rotation matrix using Rodrigues' rotation formula
K = np.array([[0, -rotation_axis[2], rotation_axis[1]],
              [rotation_axis[2], 0, -rotation_axis[0]],
              [-rotation_axis[1], rotation_axis[0], 0]])

R = np.eye(3) + np.sin(theta) * K + (1 - np.cos(theta)) * np.dot(K, K)

# Step 4: Rotate the point cloud
pcd.rotate(R, center=(0, 0, 0))

print("Displaying the point cloud after rotation...")
o3d.visualization.draw_geometries([pcd, create_coordinate_frame()])


Displaying the point cloud after rotation...
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [6]:

# Step 5: Find the centroid of the floor points
floor_points = pcd.select_by_index(inliers)
centroid = floor_points.get_center()

# Translate the point cloud to move the centroid to the origin
translation = -centroid
pcd.translate(translation)

# Step 6: Visualize the final point cloud
print("Displaying the final reoriented point cloud...")
o3d.visualization.draw_geometries([pcd, create_coordinate_frame()])


Displaying the final reoriented point cloud...


In [7]:

# Save the reoriented point cloud
#o3d.io.write_point_cloud("reoriented_point_cloud.ply", pcd)


True